# Treating Trees II
Let's consider a tree structure, similar to one described ["Why functional programming matters" by John Hughes](https://www.cs.kent.ac.uk/people/staff/dat/miranda/whyfp90.pdf), that consists of a node value followed by a sequence of zero or more child trees.  (The asterisk is meant to indicate the [Kleene star](https://en.wikipedia.org/wiki/Kleene_star).)

    tree = [] | [node tree*]

## `treestep`
In the spirit of `step` we are going to define a combinator `treestep` which expects a tree and three additional items: a base-case value `z`, and two quoted programs `[N]` and `[C]`.

    tree z [N] [C] treestep

The base-case value `z` is of some type `Z`, the `[N]` function is executed per-node and should accept the node value (whatever it is) and return a value of type `A` (which can be `Z`), and the `[C]` function should expect an `A` and a list of `Z` and return a value of type `Z`:

    z :: Z
    N :: node -> A
    C :: A [Z*] -> Z

If the current tree node is empty then just leave `z` on the stack in lieu:

       [] z [N] [C] treestep
    ---------------------------
          z

Otherwise, evaluate `N` on the node value, `map` the whole function (abbreviated here as `K`) over the child trees recursively, and then combine the result with `C`.

       [node tree*] z [N] [C] treestep
    --------------------------------------- w/ K == z [N] [C] treestep
           node N [tree*] [K] map C

## Derive the recursive form.
We can begin to derive it by finding the `ifte` stage that `genrec` will produce.  The predicate and base-case functions are trivial, so we just have to derive `J`.

    K == [not] [pop z] [J] ifte

The behavior of `J` is to accept a (non-empty) tree node and arrive at the desired outcome.

           [node tree*] J
    ------------------------------
       node N [tree*] [K] map C

So `J` will have some form like:

    J == .. [N] .. [K] .. [C] ..

Let's dive in.  First, unquote the node and `dip` `N`.

    [node tree*] uncons [N] dip
    node [tree*]        [N] dip
    node N [tree*]

Next, `map` `K` over the child trees and combine with `C`.

    node N [tree*] [K] map C
    node N [tree*] [K] map C
    node N [K.tree*]       C

So:

    J == uncons [N] dip [K] map C

Plug it in and convert to `genrec`:

    K == [not] [pop z] [J                       ] ifte
    K == [not] [pop z] [uncons [N] dip [K] map C] ifte
    K == [not] [pop z] [uncons [N] dip]   [map C] genrec

## Extract the givens to parameterize the program.
Working backwards:

    [not] [pop z]                   [uncons [N] dip] [map C] genrec
    [not] [z]         [pop] swoncat [uncons [N] dip] [map C] genrec
    [not]  z     unit [pop] swoncat [uncons [N] dip] [map C] genrec
    z [not] swap unit [pop] swoncat [uncons [N] dip] [map C] genrec
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

    TS0 == [not] swap unit [pop] swoncat

    z TS0 [uncons [N] dip]                            [map C] genrec
    z     [uncons [N] dip]                  [TS0] dip [map C] genrec
    z       [[N] dip]      [uncons] swoncat [TS0] dip [map C] genrec
    z       [N] [dip] cons [uncons] swoncat [TS0] dip [map C] genrec
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^

    TS1 == [dip] cons [uncons] swoncat

    z [N] TS1 [TS0] dip [map C]                             genrec
    z [N]               [map C]         [TS1 [TS0] dip] dip genrec
    z [N]             [C] [map] swoncat [TS1 [TS0] dip] dip genrec

The givens are all to the left so we have our definition.

## Define `treestep`

In [17]:
from notebook_preamble import D, J, V, define, DefinitionWrapper

In [30]:
DefinitionWrapper.add_definitions('''

    _treestep_0 == [not] swap unit [pop] swoncat
    _treestep_1 == [dip] cons [uncons] swoncat
    treegrind == [_treestep_1 [_treestep_0] dip] dip genrec
    treestep == [map] swoncat treegrind

''', D)

## Examples
Consider trees, the nodes of which are integers.  We can find the sum of all nodes in a tree with this function:

    sumtree == 0 [] [sum +] treestep

Running this function on an empty tree value gives zero:

       [] 0 [N] [C] treestep
    ---------------------------
          0

Running it on a non-empty node:

    [n tree*]  0 [] [sum +] treestep
    n [tree*] [0 [] [sum +] treestep] map sum +
    n [ ... ]                             sum +
    n m                                       +
    n+m


In [19]:
define('sumtree == 0 [] [sum +] treestep')

In [20]:
J('[] sumtree')  # Empty tree.

0


In [21]:
J('[23] sumtree')  # No child trees.

23


In [22]:
J('[23 []] sumtree')  # Child tree, empty.

23


In [23]:
J('[23 [2 [4]] [3]] sumtree')  # Non-empty child trees.

32


In [24]:
J('[23 [2 [8] [9]] [3] [4 []]] sumtree')  # Etc...

49


In [25]:
J('[23 [2 [8] [9]] [3] [4 []]] 0 [] [cons sum] treestep')  # Alternate "spelling".

49


In [26]:
J('[23 [2 [8] [9]] [3] [4 []]] [] [pop 23] [cons] treestep')  # Replace each node.

[23 [23 [23] [23]] [23] [23 []]]


In [27]:
J('[23 [2 [8] [9]] [3] [4 []]] [] [pop 1] [cons] treestep')

[1 [1 [1] [1]] [1] [1 []]]


In [28]:
J('[23 [2 [8] [9]] [3] [4 []]] [] [pop 1] [cons] treestep sumtree')

6


In [29]:
J('[23 [2 [8] [9]] [3] [4 []]] 0 [pop 1] [sum +] treestep')  # Combine replace and sum into one function.

6


## Redefining our BTree in terms of this form.

    BTree = [] | [[key value] left right]

What kind of functions can we write for this with our `treestep`?

The pattern for processing a non-empty node is:

    node N [tree*] [K] map C

Plugging in our BTree structure:

    [key value] N [left right] [K] map C

### Traversal
    [key value] uncons pop [left right] [K] map i
    key [value]        pop [left right] [K] map i
    key                    [left right] [K] map i
    key                    [lkey rkey ]         i
    key                     lkey rkey

This doesn't quite work:

In [14]:
J('[[3 0] [[2 0] [][]] [[9 0] [[5 0] [[4 0] [][]] [[8 0] [[6 0] [] [[7 0] [][]]][]]][]]]   23 [uncons pop] [i] treestep')

3 23 23


Doesn't work because `map` extracts the `first` item of whatever its mapped function produces.  We have to return a list, rather than depositing our results directly on the stack.


    [key value] N     [left right] [K] map C

    [key value] first [left right] [K] map flatten cons
    key               [left right] [K] map flatten cons
    key               [[lk] [rk] ]         flatten cons
    key               [ lk   rk  ]                 cons
                      [key  lk   rk  ]

So:

    [] [flatten cons] [first] treestep

In [15]:
J('[[3 0] [[2 0] [] []] [[9 0] [[5 0] [[4 0] [] []] [[8 0] [[6 0] [] [[7 0] [] []]] []]] []]]   [] [first] [flatten cons] treestep')

[3 2 9 5 4 8 6 7]


There we go.

### In-order traversal with `treestep`.

From here:

    key [[lk] [rk]] C
    key [[lk] [rk]] i
    key  [lk] [rk] roll<
    [lk] [rk] key swons concat
    [lk] [key rk]       concat
    [lk   key rk]

So:

    [] [i roll< swons concat] [first] treestep

In [16]:
J('[[3 0] [[2 0] [] []] [[9 0] [[5 0] [[4 0] [] []] [[8 0] [[6 0] [] [[7 0] [] []]] []]] []]]   [] [uncons pop] [i roll< swons concat] treestep')

[2 3 4 5 6 7 8 9]


## with `treegrind`?
What kind of functions can we write for this with our `treegrind`?

The pattern for processing a non-empty node is:

    node N [tree*] [K] C

Plugging in our BTree structure:

    [key value] N [left right] [K] C

In [33]:
J('[23 [44] [18] ] "z" ["N"] ["C"] treegrind')

23 'N' [[44] [18]] [[not] [pop 'z'] [uncons ['N'] dip] ['C'] genrec] 'C'


In [35]:
J('[[3 0] [[2 0] [] []] [[9 0] [[5 0] [[4 0] [] []] [[8 0] [[6 0] [] [[7 0] [] []]] []]] []]]   23 ["N"] [step] treegrind')

[3 0] 'N' [2 0] 'N' 23 23 [9 0] 'N' [5 0] 'N' [4 0] 'N' 23 23 [8 0] 'N' [6 0] 'N' 23 [7 0] 'N' 23 23 23 23


In [36]:
J('0 [[3 0] [[2 0] [] []] [[9 0] [[5 0] [[4 0] [] []] [[8 0] [[6 0] [] [[7 0] [] []]] []]] []]]   23 [first ] [step] treegrind')

0 3 2 23 23 9 5 4 23 23 8 6 23 7 23 23 23 23


In [39]:
J('0 [[3 0] [[2 0] [] []] [[9 0] [[5 0] [[4 0] [] []] [[8 0] [[6 0] [] [[7 0] [] []]] []]] []]]   [] [first ] [map cons] treegrind')

0 [3 [2 [] []] [9 [5 [4 [] []] [8 [6 [] [7 [] []]] []]] []]]
